In [1]:
# This Youtube video walks through this notebook
from IPython.display import YouTubeVideo
YouTubeVideo('xjFtLF95uBc')

In [2]:
from jove.DotBashers import *
from jove.Def_md2mc  import *
from jove.Def_NFA    import *

In [3]:
nfa_ends0101 = md2mc(src="File", fname="nfafiles/endsin0101.nfa")

In [4]:
nfa_ends0101

In [5]:
dotObj_nfa(nfa_ends0101)

In [6]:
dotObj_nfa(nfa_ends0101, FuseEdges=True)

In [7]:
NFA_fig61ab = { 'Q': {'I', 'S0', 'S1', 'S2', 'F'},
                'Sigma': {'0', '1'},
                'Delta': { # 
                             ('I', '0')  : {'I'},
                             ('I', '1')  : {'I', 'S0'},
                             ('I', '')   : {'S0'},
                             #
                             ('S0', '1') : {'S1'},
                             #
                             ('S1', '0') : {'S2'},
                             ('S1', '1') : {'S2'},
                             #
                             ('S2', '0') : {'F'},
                             ('S2', '1') : {'F'},
               },
                  'Q0': {'I'}, 
                  'F' : {'F'}   
                }
NFA_fig61ab

In [8]:
step_nfa(NFA_fig61ab, "I", "")

In [9]:
step_nfa(NFA_fig61ab, "I", "0")

In [10]:
step_nfa(NFA_fig61ab, "I", "1")

In [11]:
# NFA for ((aa)+(bbb)+)+
NFA23 = { 'Q': {'A0','A1','B0','B1','B2','F'},
          'Sigma': {'0', '1'},
          'Delta': { # 
                     ('A0', '0') : {'A1'},
                     ('A1', '0') : {'B0'},
                     #
                     ('B0', '')  : {'A0'},
                     #
                     ('B0', '1') : {'B1'},
                     ('B1', '1') : {'B2'},
                     ('B2', '1') : {'F'},
                     # 
                     ('F', '')   : {'B0'},
                   },
          'Q0': {'A0'}, 
          'F' : {'F'}   
        }
Source(dot_nfa(NFA23))

In [12]:
Source(dot_nfa(NFA23, visible_eps=True))

In [13]:
NFA23["Q0"]

In [14]:
run_nfa(NFA23, NFA23["Q0"], '0', True)

In [15]:
Source(dot_nfa(NFA_fig61ab))

In [16]:
run_nfa(NFA23, set({'A0'}), '', True)

In [17]:
Eclosure(NFA_fig61ab, {"I"})

In [18]:
Eclosure(NFA_fig61ab, {"S0"})

In [19]:
run_nfa(NFA_fig61ab,{"I"},"")

In [20]:
run_nfa(NFA_fig61ab,{"I"},"", True)

In [21]:
run_nfa(NFA_fig61ab,{"I"},"0")

In [22]:
run_nfa(NFA_fig61ab,{"I"},"1")

In [23]:
run_nfa(NFA_fig61ab,{"I"},"100")

In [24]:
run_nfa(NFA_fig61ab,{"I"},"100", True)

In [25]:
run_nfa(NFA_fig61ab,{"I"},"00110")

In [26]:
run_nfa(NFA_fig61ab,{"I"},"00110", True)

In [27]:
accepts_nfa(NFA_fig61ab, "")

In [28]:
accepts_nfa(NFA_fig61ab, "", True)

In [29]:
accepts_nfa(NFA_fig61ab, "0", True)

In [30]:
accepts_nfa(NFA_fig61ab, "100", True)

In [31]:
dotObj_nfa(NFA23, visible_eps=True, nfaName="NFA23")

In [32]:
dotObj_nfa(NFA23, visible_eps=False, nfaName="NFA23")

In [33]:
n2DFA23 = nfa2dfa(NFA23)
dotObj_dfa(n2DFA23, "n2dNFA23")

In [34]:
n2DFA61 = nfa2dfa(NFA_fig61ab)
dotObj_dfa(n2DFA61, "n2dNFA_fig61ab")

In [35]:
dotObj_nfa(NFA_fig61ab,visible_eps=False,nfaName="NFA_fig61ab")

In [36]:
dotObj_nfa(NFA_fig61ab,visible_eps=True,nfaName="NFA_fig61ab")

In [37]:
dotObj_dfa(nfa2dfa(NFA_fig61ab), 'n1')

# Brzozowski's DFA Minimization

Picking up from our earlier discussions, to minimize a DFA using Brzozowski's algorithm, here are the steps:

* Make sure that the given DFA has no unreachable states
* Reverse the DFA
* Determinize it
* Reverse that DFA
* Determinize it

Thus we need to write a routine to reverse a DFA. We already have a way to ensure that a DFA does not have unreachable states (in another Jupyter notebook; we won't bother to include it here, and trust the user to always provide such DFA only).

We can observe that if a DFA has black-hole states, then those states won't matter in the reversed machine (reversed NFA). Thus, we can work with __partial__ dfa (i.e., DFA that are partially consistent).

## DFA reversal

In [38]:
def inSets(D,trg,ch):
    """In : D   = partially consistent dfa,
            trg = a target state in D["q"]
            ch  = a member of D["Sigma"]
       Out: a set of states. { q s.t. Delta[q,ch] == trg }
    """
    return { q for q in D["Q"] if D["Delta"][(q,ch)] == trg }

def rev_dfa(D):
    """In : D = a partially consistent DFA without any unreachable states.
       Out: A consistent NFA whose language is D's language reversed.
    """
    # 1. Given that NFAs start from a SET of states, we already have that
    #   info. No need to add any transitions from "a new initial state" 
    #   etc
    
    # 2. Now add the inSets of each state as the NFA next set of states
    NDict = { (q,ch) : inSets(D,q,ch) 
              for q in D["Q"] 
              for ch in D["Sigma"] }
    
    # Notice that we retain D["Q"] and start from Q0 = D["F"]
    # going backwards along NDict toward F_dfa = { D["q0"] }
    return mk_nfa(D["Q"], D["Sigma"], NDict, D["F"], {D["q0"]})

In [39]:
DFA34 = { 'Q': {'A', 'IF', 'B'},
          'Sigma': {'0', '1'},
          'Delta': { ('IF', '0'): 'A',
                     ('IF', '1'): 'IF',
                     ('A', '0'): 'B',
                     ('A', '1'): 'A',
                     ('B', '0'): 'IF',
                     ('B', '1'): 'B' },
          'q0': 'IF', 
          'F': {'IF'}   
        }
dotObj_dfa(DFA34, "DFA34")

In [40]:
DFA34_rev = rev_dfa(DFA34)
dotObj_nfa(DFA34_rev, "DFA34_rev")

In [41]:
DFA34_rev_det = nfa2dfa(DFA34_rev)
dotObj_dfa(DFA34_rev_det, "DFA34_rev_det")

In [42]:
DFA34_rev_det_rev = rev_dfa(DFA34_rev_det)
dotObj_nfa(DFA34_rev_det_rev, "DFA34_rev_det_rev")

In [43]:
DFA34_rev_det_rev_det = nfa2dfa(DFA34_rev_det_rev)
dotObj_dfa(DFA34_rev_det_rev_det, "DFA34_rev_det_rev_det")

__TRY NEW EXAMPLE HERE__

In [44]:
dotObj_dfa(n2DFA23, "n2DFA23")

In [45]:
rev_n2DFA23 = rev_dfa(n2DFA23)
dotObj_nfa(rev_n2DFA23, "rev23")

In [46]:
det_rev_n2DFA23 = nfa2dfa(rev_dfa(n2DFA23))
dotObj_nfa(rev_n2DFA23, "rev23")

In [47]:
n2DFA23
dotObj_dfa(nfa2dfa(rev_dfa(nfa2dfa(rev_dfa(n2DFA23)))), "rdrd")

In [48]:
D34bl = { 'Q': {'A', 'IF', 'B', 'A1', 'B1'},
          'Sigma': {'0', '1'},
          'Delta': { ('IF', '0'): 'A',
                     ('IF', '1'): 'IF',
                     ('A', '0'): 'B1',
                     ('A', '1'): 'A1',
                     ('A1', '0'): 'B',
                     ('A1', '1'): 'A',
                     ('B1', '0'): 'IF',
                     ('B1', '1'): 'B',
                     ('B','0') : 'IF',
                     ('B', '1'): 'B1' },
          'q0': 'IF', 
          'F': {'IF'}   
        }

dotObj_dfa(D34bl, "D34bl")

In [49]:
dotObj_dfa(nfa2dfa(rev_dfa(nfa2dfa(rev_dfa(D34bl)))), "D34bl_rdrd")

In [50]:
nfaMultiQ0 = md2mc('''
NFA
I0 : a | b | c -> A, B
I0 : c -> F
I1 : a | b -> A, B
A  : c -> F
B  : d -> F
''')

In [51]:
dotObj_nfa(nfaMultiQ0)

In [52]:
dotObj_nfa(nfaMultiQ0, FuseEdges=True)

In [53]:
dfaMQ0 = nfa2dfa(nfaMultiQ0)

In [54]:
dotObj_dfa(dfaMQ0)

In [55]:
dotObj_dfa(dfaMQ0, FuseEdges=True)

In [56]:
dotObj_nfa(rev_dfa(dfaMQ0))

In [57]:
dotObj_nfa(rev_dfa(dfaMQ0), FuseEdges=True)

In [58]:
help(min_dfa_brz)

In [59]:
dotObj_dfa(dfaMQ0)

In [60]:
dotObj_dfa(min_dfa_brz(dfaMQ0))

In [61]:
thirdlast1alt = md2mc('''
NFA
I : 0   -> I    !! On input 0 in the init state I, stay in I
                !! On input 1, fork two paths, one staying in I
                !! and the other going to state A.
I : 1   -> I,A  !! Threads that land in state A must enter 
                !! the final state in two more steps. 
A : 0|1 -> B    !! A move out of F kills this speculative 
B : 0|1 -> F    !! token, but more tokens may be on their way
''')

In [62]:
dotObj_nfa(thirdlast1alt)

In [63]:
blimp = md2mc('''
DFA 
I1 : a -> F2
I1 : b -> F3
F2 : a -> S8
F2 : b -> S5
F3 : a -> S7
F3 : b -> S4
S4 : a | b -> F6
S5 : a | b -> F6
F6 : a | b -> F6
S7 : a | b -> F6
S8 : a -> F6
S8 : b -> F9
F9 : a -> F9
F9 : b -> F6
''')

In [64]:
DOblimp = dotObj_dfa(blimp)

In [65]:
DOblimp

In [66]:
dotObj_dfa(blimp, True)

In [67]:
rblimp = rev_dfa(blimp)
DOrblimp = dotObj_nfa(rblimp)
DOrblimp

In [68]:
dotObj_nfa(rblimp, FuseEdges=True)

In [69]:
DOrblimp.source

In [70]:
drblimp = nfa2dfa(rblimp)
drblimp

In [71]:
DOdrblimp = dotObj_dfa(drblimp)
DOdrblimp

In [72]:
DOdrblimp.source

In [73]:
rdrblimp = rev_dfa(drblimp)
DOrdrblimp = dotObj_nfa(rdrblimp)
DOrdrblimp

In [74]:
DOrdrblimp.source

In [75]:
drdrblimp = nfa2dfa(rdrblimp)
DOdrdrblimp = dotObj_dfa(drdrblimp)
DOdrdrblimp

In [76]:
DOdrdrblimp.source

In [77]:
nfahas01 = md2mc('''
NFA 
I : 0 | 1 -> I
I : '' -> A
A : 0  -> B
B : 1  -> C
C : 0 | 1 -> C
C : '' -> F
''')
DOhas01 = dotObj_nfa(nfahas01)
DOhas01

In [78]:
dfahas01 = nfa2dfa(nfahas01)
DOdfahas01 = dotObj_dfa(dfahas01)
DOdfahas01